# Dynamically update agents

Often we want to dynamically update the instance of the model being used by our agent, rather than just the state or context available to the agent. Specifically this is when we want to update one or more of:
- The foundational model (LLM)
- System prompt for the model
- Tools available to the model

We do this my modifying the `ModelRequest` object inside a function decorated with `wrap_model_call`

### ModelRequest
The `ModelRequest` object exposes:

- `request.messages` - conversation history
- `request.state` - current agent state
- `request.runtime` - runtime context (including custom defined context)
- `request.system_message` - the system prompt
- `request.override(...)` - returns a new request with modified fields (`tools`, `model`, `system_message`, etc.)
  - Modify underlying foundation model: `request.override(model=<new model>)`
  - Modify tools available to model: `request.override(tools=[...])`

### wrap_model_call
`wrap_model_call` is a wrap-style hook that intercepts every model call in the agent loop, allowing you to control whether and how the handler is called:

- **Zero times**: Short-circuit (return cached/mock response)
- **Once**: Normal flow (possibly with modified request)
- **Multiple times**: Retry logic

### Use cases for dynamic model updates

1. **Dynamic model selection** base on task domain, task complexity, tool requirements, etc.
2. **Dynamic tool selection** for role-based access control, tool gating based on conversation phase, semantic tool selection (model powered)
3. **Dynamic system message modification** for runtime context injection
4. **Dynamic selection of structured output model**

### Wrap-style vs. Node-style Middleware vs. Tool Calls

| **Wrap-style** | **Node-style** | **Tool Calls** |
| -------------- | -------------- | -------------- |
| `ModelRequest` | `State` + `Runtime` | `ToolRuntime` |
| Adjust an agent's tools, prompt, model while running | Adjust an agent's state while running | Agent adjusts its own state or retires runtime context |
| e.g. updating available tools based on context | e.g. Summarizing messages from long conversations | e.g. Updating state with user provided data |


In [ ]:
from collections.abc import Callable
from typing import Literal

from langchain.agents import create_agent
from langchain.agents.middleware import ModelRequest, ModelResponse, wrap_model_call
from langchain.chat_models import init_chat_model
from langchain.messages import HumanMessage
from langchain.tools import BaseTool, tool
from loguru import logger
from pydantic import BaseModel, Field

from chain_reaction.config import APIKeys, ModelBehavior, ModelName
from chain_reaction.utils import get_last_message

api_keys = APIKeys()


# Initialize a chat model
chat_model = init_chat_model(
    model=ModelName.CLAUDE_HAIKU,
    timeout=None,
    max_retries=2,
    api_key=api_keys.anthropic,
    **ModelBehavior.deterministic().model_dump(),
)

In [ ]:
# Define model for user context
type UserRole = Literal["admin", "developer", "user"]


class UserContext(BaseModel):
    """Context schema for user data."""

    role: UserRole = Field(description="The role of the user making the request.")


# Define mock tools that can be enabled/disabled based on user role
@tool
def search_documentation(query: str) -> list[str]:
    """Search the documentation for a given query.

    Args:
        query: The search query string.

    Returns:
        list[str]: A list of documentation results.
    """
    # Mock implementation of documentation search
    logger.debug(f"Searching documentation for query: {query}")
    return ["doc1", "doc2", "doc3"]


@tool
def search_database(query: str) -> list[str]:
    """Search the internal database for a given query.

    Args:
        query: Query string.

    Returns:
        list[str]: A list of database results.
    """
    # Mock implementation of database search
    logger.debug(f"Searching database for query: {query}")
    return ["record1", "record2", "record3"]


@tool
def update_record(record: str) -> bool:
    """Update a record in the internal database.

    Args:
        record: The record to update.

    Returns:
        bool: True if the update was successful, False otherwise.
    """
    # Mock implementation of record update
    logger.debug(f"Updating record: {record}")
    return True


# Define middleware to update available tools based on user role
@wrap_model_call
def update_available_tools(
    request: ModelRequest,
    handler: Callable[[ModelRequest], ModelResponse],
) -> ModelResponse:
    """Middleware to update tools available to an agent based on user role."""
    # Extract user context from the request
    if not isinstance(context := request.runtime.context, UserContext):
        logger.error("User context is missing or of incorrect type.")
        raise RuntimeError("User context is required for tool selection.")

    # Set available tools based on user role
    tools: list[BaseTool]
    if context.role == "admin":
        logger.debug("User role is admin; enabling all tools.")
        tools = [search_documentation, search_database, update_record]
    elif context.role == "developer":
        logger.debug("User role is developer; enabling documentation and database search tools.")
        tools = [search_documentation, search_database]
    elif context.role == "user":
        logger.debug("User role is user; enabling only documentation search tool.")
        tools = [search_documentation]
    else:
        logger.warning(f"Unknown user role: {context.role}; disabling all tools.")
        tools = []

    # Override the tools in the request
    request = request.override(tools=tools)

    # Pass the modified request to the handler to run the updated model request
    return handler(request)


# Create an agent with the middleware
agent = create_agent(
    model=chat_model,
    tools=[
        # NOTE: we must initialize agent with all possible tools; middleware will filter them
        search_documentation,
        search_database,
        update_record,
    ],
    middleware=[update_available_tools],
    context_schema=UserContext,
)

In [ ]:
# Invoke the agent with different user roles
response = agent.invoke(
    {"messages": [HumanMessage(content="Find information about API usage in the documentation.")]},
    context=UserContext(role="user"),
)

print("Agent Response:", get_last_message(response).content)

In [ ]:
# Invoke the agent with different user roles
response = agent.invoke(
    {"messages": [HumanMessage(content="Find information about my purchases in the database.")]},
    context=UserContext(role="user"),
)

print("Agent Response:", get_last_message(response).content)